In [10]:
import json
from collections import Counter
from keras.utils import to_categorical
import numpy as np
def get_data(filename):
    with open(filename) as f:
        list=json.load(f)
    return list
def counter(note):
    counters=Counter(note)
    note_count=sorted(counters.items(),key=lambda x:-x[1])
    notes,_=zip(*note_count)
    note_to_id={note:id for id,note in enumerate(notes)}
    return note_to_id
note=get_data('origin.json')
fin=counter(note)
X_train=[]
Y_train=[]
sequence_batch=100
for i in range(len(note)-sequence_batch):
    X_pre=note[i:i+sequence_batch]
    Y_pre=note[i+sequence_batch]
    X_train.append([fin[note]for note in X_pre])
    Y_train.append(fin[Y_pre])
X_train=X_train[:10000]
Y_train=Y_train[:10000]
X_one_hot = to_categorical(X_train)
Y_one_hot = to_categorical(Y_train)


In [11]:
import keras
from keras.callbacks import ModelCheckpoint
from keras.models import Input, Model
from keras.layers import  Dropout, Dense,LSTM 
from keras.optimizers import Adam
from keras.utils import plot_model
# X_one_hot.shape[1:] = (100, 308)
input_tensor = Input(shape=X_one_hot.shape[1:])
lstm = LSTM(512,return_sequences=True)(input_tensor)
dropout = Dropout(0.3)(lstm)

lstm = LSTM(256)(dropout)
dropout = Dropout(0.3)(lstm)
dense = Dense(Y_one_hot.shape[-1], activation='softmax')(dropout)

model = Model(inputs=input_tensor, outputs=dense)
optimizer = Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
model.summary()


Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 100, 1147)         0         
_________________________________________________________________
lstm_11 (LSTM)               (None, 100, 512)          3399680   
_________________________________________________________________
dropout_11 (Dropout)         (None, 100, 512)          0         
_________________________________________________________________
lstm_12 (LSTM)               (None, 256)               787456    
_________________________________________________________________
dropout_12 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 1147)              294779    
Total params: 4,481,915
Trainable params: 4,481,915
Non-trainable params: 0
_________________________________________________

In [12]:
filepath = "./{epoch}--weights{loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(
    filepath,
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)
callbacks_list = [checkpoint]
model.fit(X_one_hot, Y_one_hot, epochs=20, batch_size=2048,callbacks=callbacks_list)

Epoch 1/20
10000/10000 [==============================] - 355s 35ms/step - loss: 6.9786 - accuracy: 0.0141
Epoch 2/20
10000/10000 [==============================] - 367s 37ms/step - loss: 5.9972 - accuracy: 0.0178
Epoch 3/20
 6144/10000 [=================>............] - ETA: 2:37 - loss: 5.3634 - accuracy: 0.0184